In [ ]:
import os
import sys
import pickle

import sys
import commonutils
from commonutils import ModelResults

import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
#from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from dataclasses import dataclass

from sklearn.cross_decomposition import PLSRegression
import warnings

from sklearn import preprocessing

from copy import deepcopy
import pickle

import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from commonutils import ModelsStore
warnings.simplefilter("ignore")

from commonfuncsforcli import *

sys.path.append('./CLossLr')
import customlosslr as clr



In [ ]:
rootdir = './Results'
functional = "PBE"
basisset = "MINIX"
models = {}
for dir in os.listdir(rootdir):
    if os.path.isdir(os.path.join(rootdir, dir)):
        if dir == functional+"_"+basisset:
            for file in os.listdir(os.path.join(rootdir, dir)):
                if file.endswith('.pkl'):
                    print(f'Loading model from {os.path.join(rootdir, dir, file)}')
                    with open(os.path.join(rootdir, dir, file), 'rb') as f:
                        model = pickle.load(f)
                        modelname = file.replace('.pkl', '')
                        print(f'  Loaded model {modelname}')
                        models[modelname] = model
   


In [ ]:
equations = {"Te": "Kinetic_Energy", \
         "V_NN": "Nuclear_Repulsion",\
         "V_eN": "One_Electron_Energy - Kinetic_Energy",\
         "EX": "EX",\
         "E_J": "Two_Electron_Energy - EX - EC",\
         "DC": "Dispersion_correction",\
         "EC": "EC"}
SHIFTFT = "DC"

selected_basisset = "SVP"
selected_functional = "PBE"
REMOVEFLPSFROMTRAINING = False

IMPORTMODELTOSTART = False

functionals = []
basis_sets = []

print("Selected functional: ", selected_functional)
print(" Selected basis set: ", selected_basisset)
print("        Functionals: ", functionals)
print("         Basis sets: ", basis_sets)

featuresvalues_perset,\
fullsetnames, \
models_results, \
supersetnames = readdata(shiftusingFT=SHIFTFT, \
                selected_functionalin=selected_functional, \
                selected_basisin=selected_basisset, \
                equations=equations)

sep = "_"
for setname in fullsetnames:
    desciptors = {}
    k = selected_functional + sep +\
                selected_basisset 
    for features in featuresvalues_perset[setname]:
        for val in features:
            if val.find(k) != -1:
                if val not in desciptors:
                    desciptors[val] = [features[val]]
                else:
                    desciptors[val].append(features[val])

    for features in featuresvalues_perset[setname]:
        for val in features:
            for func in functionals:
                for basis in basis_sets:
                    if not(basis == selected_basisset and \
                            func == selected_functional):
                        if val.find(func + sep + basis) != -1:
                            actualk = val 
                            refk  = selected_functional + sep  + \
                                    selected_basisset + \
                                    val.replace(func + sep + basis, "")
                            newk = actualk + "_difftoref"
                            if newk not in desciptors:
                                desciptors[newk] = [features[actualk]-features[refk]]
                            else:
                                desciptors[newk].append(features[actualk]-features[refk])
    
    models_results[setname].features = desciptors

# features selection
setname = "Full"
numoffeat = len(models_results[setname].features)
print("Number of features for ", numoffeat)
for setname in fullsetnames:
    if len(models_results[setname].features) != numoffeat:
        print("Number of features for ", setname, " is different")
        sys.exit(1)

toremove = []
setname = "Full"
for k in models_results[setname].features:
    if len(set(models_results[setname].features[k])) == 1:
        toremove.append(k)
        print("Constant fatures to remove: ", k)

# remove constant values
for setname in fullsetnames:
    for k in toremove:
        del models_results[setname].features[k]

# force removing features Nuclear Repulsion difference
tormfeatname = set()
print("Removing Nuclear Repulsion differences")
for setname in fullsetnames: 
    toremove = []
    for k in models_results[setname].features:
        if k.find("NR_difftoref") != -1:
            toremove.append(k)
    for k in toremove:
        #print("Removing feature ", k)
        tormfeatname.add(k)
        del models_results[setname].features[k]

print("Removed features: ", tormfeatname)
setname = "Full"
numoffeat = len(models_results[setname].features)
print("Number of features ", numoffeat)
for setname in fullsetnames:
    if len(models_results[setname].features) != numoffeat:
        print("Number of features for ", setname, " is different")
        sys.exit(1)